<a href="https://colab.research.google.com/github/keulreobeu/deep_learning_for_the_life_sciences/blob/main/%EB%8F%85%EC%84%B1_%EB%B6%84%EC%9E%90_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

런타임 구버전(25.07)으로 바꾸기

In [ ]:
!pip install --pre deepchem

런타임 재시작 후 다시실행


In [ ]:
%pip install -U pip
%pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
import deepchem as dc
from rdkit import Chem
import numpy as np
import pandas as pd

독성 분자 데이터셋

#load_tox21()
*    featurizer: Featurizer | str = 'ECFP', # 피처화 방식 선택 ECFP, Graphconv, weave, smiles2lmg, MolGanFeaturizer 등
*    splitter: Splitter | str | None = 'scaffold', # random, index, scaffold
*    transformers: List[TransformerGenerator | str] = ['balancing'], # 클래스 불균형, 특성 정규화 등을 적용, balancing, normalization 등
*    reload: bool = True, # bool 이미 전처리된 데이터가 있으면 재사용 가능
*    data_dir: str | None = None,# 원본 데이터 저장 경로
*    save_dir: str | None = None, # 전처리 후 데이터 캐시 경로
*    tasks: List[str] = ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53'], **kwargs → Tuple[List[str], Tuple[Dataset, ...], List[Transformer]] # 타겟 이름 그리고 반환 방식


---


#Featurizer

*    ECFP: [CircularFingerprint], 고정 길이의 비트 백터 생성, 주변 원자 구조을 순환적으로 탐색하여 특싱을 생성
*    GraphConv: [ConvMolFeaturizer or MolGraphConvFeaturizer], 분자 그래프를 보고, 원자 노드와 결합 정보를 이용, 각 원자의 특성 백터, 이웃 관계, 결합 타입 등을 포함하여 백터를 생성. 구조적 정보가 반영됨
*    Weave: [WeaveFeaturizer], GraphConv쪽보다 복잡한 pair-wise특징 정보 포함, 원자-원자 쌍마다 관계를 계산하여 반영,
*    Smiles2Img: [SmilesToImage], SMILES 문자열을 2D 이미지 형테로 변환하여 CNN등의 이미지 모델 입력으로 사용, 이미지 기반 신경망 이용시 사용
*    MOlGanFeaturizer: [MolGanFeaturizer], MolGAN(생성 모델)용 특징, 그래프 매트릭스 혹은 adjacency + atom/bond type 정보기반 표현

    
---


# 추가 학습 정보
*    pair-wise: 대부부의 결합이 많아야 두개 요소의 상호작용에 의해 야기된다는 관찰에 기반한 기법
*    adjacency type: 각 원자의 화학적 특성을 표현하는 특징 백터, 원자 번호, 원자가 속한 족/주기 정보, 전자쌍, 형식 전하, 방향족성 여부, 수소 수, 극성/비극성 여부 등
*    Bond type: 결합 종류를 표현, 당일, 이중, 삼중, 방향족 결합을 표현, 일부 모델은 겨합 방향, 결합 길이등 정보를 백터에 포함



---

# 각 피쳐 정보
*   'NR-AR': 안드로겐 수용체 활성 여부
*   'NR-AR-LBD': 안드로겐 수용체의 리간드 결합 도메인 활성화 여부
*   'NR-AhR': 아릴 탄화수소 수용체 활성 여부
*   'NR-Aromatase': 아로마타제 효소 억제 활성 여부
*   'NR-ER': 에스트로겐 수용체 활성 여부
*   'NR-ER-LBD': 에스트로겐 수용체의 리간드 결합 도메인 활성 여부
*   'NR-PPAR-gamma': 핵수용체 PPAR-gamma의 리간드 활성 여부
*   'SR-ARE': 항산화 반응 요소 활성화 여부
*   'SR-ATAD5': DNA 손상 / 복구 관련 단백질 ATAD5 활성화 여부
*   'SR-HSE': 열충격 요소 활성화 여부
*   'SR-MMP': 미토콘드리아의 막전위 손상 여부
*   'SR-p53': 종양 억제 단백질 p-53 활성화 여부

In [ ]:
tasks, datasets, transformers = dc.molnet.load_tox21()
train_dataset, valid_dataset, test_dataset = datasets

In [ ]:
x = train_dataset.X
y = train_dataset.y

In [ ]:
train_dataset.to_dataframe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,w4,w5,w6,w7,w8,w9,w10,w11,w12,ids
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.045415,1.146078,1.056211,1.025552,1.172679,1.035385,1.055765,1.174650,1.052884,CC(O)(P(=O)(O)O)P(=O)(O)O
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.045415,1.146078,1.056211,1.025552,0.000000,1.035385,1.055765,1.174650,1.052884,CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.045415,1.146078,1.056211,1.025552,1.172679,1.035385,1.055765,0.000000,1.052884,OC[C@H](O)[C@@H](O)[C@H](O)CO
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.172679,0.000000,1.055765,0.000000,0.000000,CCCCCCCC(=O)[O-].CCCCCCCC(=O)[O-].[Zn+2]
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.045415,1.146078,1.056211,1.025552,1.172679,1.035385,1.055765,1.174650,1.052884,CC(C)COC(=O)C(C)C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6259,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.172679,0.000000,1.055765,0.000000,0.000000,CC1CCCCN1CCCOC(=O)c1ccc(OC2CCCCC2)cc1
6260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.172679,0.000000,1.055765,0.000000,0.000000,Cc1cc(CCCOc2c(C)cc(-c3noc(C(F)(F)F)n3)cc2C)on1
6261,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.045415,1.146078,1.056211,1.025552,0.000000,1.035385,1.055765,6.725738,1.052884,O=C1OC(OC(=O)c2cccnc2Nc2cccc(C(F)(F)F)c2)c2ccc...
6262,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.045415,1.146078,1.056211,1.025552,1.172679,1.035385,1.055765,6.725738,1.052884,CC(=O)C1(C)CC2=C(CCCC2(C)C)CC1C


# 기본적인 경사하강법을 이용하여 간단한 머신러닝

In [ ]:
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])
model.fit(train_dataset, nb_epoch=10)

0.4951243718465169

In [ ]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(valid_dataset, [metric], transformers)

In [ ]:
print(train_scores)
print(test_scores)

{'mean-roc_auc_score': 0.9587619833910873}
{'mean-roc_auc_score': 0.7120695676774208}


bool 타입의 결과라 0.5가 기준 그리고 독성이 없는 경우가 90%라 목표 auc는 97%이상은 되어야 한다 생각함

---

#EDA
## 데이터
*   데이터에 NaN데이터가 많음
*   각 피쳐들은 세포가 스트레스를 받을때 일어나는 반응들이나 세포 스스로 자가수복이 가능하거나 일반적인 환경 스트레스에도 일어날 수 있는 피쳐들이 많음 -> 피쳐수가 너무 적은 경우 데이터를 제거하는 것도 고려,
*   단순 활성이 아닌 각 활성마다 활성 정도를 반영해야함 ex) DNA 손상 단백질 활성만 매우 높다 -> DNA파괴가 심각함 -> 세포독성
*   하지만 tox21 데이터에는 이에 대한 정보가 없다.
*   세포독성의 정도 같은 구체적인 데이터 또한 없다
*   분자의 화학구조 표현식이 존재 -> 분자의 반응성 정도를 반영 가능


## 머신런닝
*   일반적으로 독성이 있으려면 분자의 반응성이 높아야 한다
*    -> 전자의 과 또는 부족(전기음성도), 이온과 같은 공명 안전화가 가능한 중간체 존재
*   약한 결합력, 불포화 결합, 삼각형 또는 사각형 과 같은 불안정한 구조
*   등등 분자의 구조가 반응성에 영향을 준다.

-> GraphConv같은 분자구조를 반영할 수 있는 방식을 이용해야함



----> 4장의 내용